In [ ]:
# default_exp dashboards

# dashboards

> Supplies dashboards to investigate datasets and training results. Dashboards are defined as classes, to show the dashboard use the .show() function on an dashboard instance.

In [ ]:
#export
from typing import Union

from bokeh.plotting import show, output_notebook
from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter
from bokeh.models import ColumnDataSource

import panel as pn
import panel.widgets as pnw
import pandas as pd

from icevision_dashboard.utils import *
from icevision_dashboard.components import *

In [ ]:
pn.extension()

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()

## Dataset overview

In [ ]:
#export
class DatasetOverview:
    def __init__(self, records=None, data=None, class_map=None, height=500, width=1500):
        """If records is None no gallery will be generated. If the data is generated with aggregate_record_data and class map is used the class map needs to be provided as well."""
        if records is None and data is None:
            raise ValueError("Either records or data needs to be given.")
        self.records = records
        self.class_map = class_map
        if data is None:
            self.data = pd.DataFrame(aggregate_record_data(records, class_map))
        else:
            self.data = data
        self.height = height
        self.width = width
        
    def _generate_dataset_tab(self):
        overview_table = table_from_dataframe(self.data, width=self.width, height=self.height)
        return pn.Column(overview_table)
        
    def _generate_image_tab(self):
        overview_table = table_from_dataframe(calculate_image_stats(self.data), width=self.width, height=50)
        if self.records is None:
            img_gallery = None
        else:
            img_gallery = gallery(self.records, self.class_map, height=self.height-20)
        return pn.Column(pn.Row(overview_table, align="center"), pn.Row(img_gallery, align="center"))
        
    def _generate_class_tab(self):
        overview_table = table_from_dataframe(calculate_class_stats(self.data), height=150)
        mixing_matrix_df, mapping = calculate_mixing_matrix(self.data, mixing_col="filepath", mixing_objects="label")
        mixing_histogram = histogram_2d(mixing_matrix_df, "row_name", "col_name", "values", height=self.height-150, width=self.height-150)
        stacked_annotations_per_image_hist = annotations_per_image_stacked_hist(self.data, height=self.height-150, width=self.height-150)
        return pn.Column(pn.Row(overview_table, align="center"), pn.Row(mixing_histogram, stacked_annotations_per_image_hist, align="center"), align="center")
        
    def show(self):
        dataset_tab = self._generate_dataset_tab()
        image_tab = self._generate_image_tab()
        class_tab = self._generate_class_tab()
        
        return pn.Tabs(("Dataset overview", dataset_tab), ("Image stats", image_tab), ("Class stats", class_tab))

In [ ]:
dso = DatasetOverview(test_valid_records, class_map=test_class_map, height=700)
dso.show()

In [ ]:
dso = DatasetOverview(data=aggregate_record_data(test_valid_records, test_class_map), class_map=test_class_map, height=700)
dso.show()

In [ ]:
#export
class MultiDatasetOverview:
    def __init__(self, datasets, height=500, width=1500):
        self._datasets = datasets
        self._observer = []
        self.width = width
        self.height = height
    
    def __iter__(self):
        for item in self.datasets:
            yield item
    
    def register_callback(self, callback):
        self._observer.append(callback)
    
    def _trigger_observer(self):
        for observer in self._observer:
            observer(self._datasets)
    
    def append(self, item):
        self._datasets.append(item)
        self._trigger_observer()
        
    def remove(self, item):
        self._datasets.remove(item)
        self._trigger_observer()
        
    def insert(self, index, item):
        self._datasets.insert(index, item)
        self._trigger_observer()
    
    def pop(self, index=-1):
        poped_item = self._datasets.pop(index)
        self._trigger_observer()
        return poped_item
    
    def extend(self, iterable):
        self._datasets.extend(iterable)
        self._trigger_observer()
    
    @property
    def datasets(self):
        return self._datasets
    
    @datasets.setter
    def datasets(self, value):
        self._datasets = value
        self._trigger_observer()
        
    def create_datasets_overview(self):
        dataset_stats_list = []
        for dataset in self._datasets:
            dataset_stats = calculate_dataset_stats(dataset)
            dataset_stats["classes"] = ", ".join(dataset_stats["classes"])
            dataset_stats_list.append(dataset_stats)
        dataset_stats_df = pd.DataFrame(dataset_stats_list)
        
        template = """<span href="#" data-toggle="tooltip" title="<%= value %>"><%= value %></span>"""
        table = pnw.DataFrame(dataset_stats_df, formatters={"classes": HTMLTemplateFormatter(template=template)}, selection=[0])
        return table
    
    def show(self):
        del_button = pnw.Button(name="Delete")
        datasets_overview = self.create_datasets_overview()
        dataset_overview = DatasetOverview(data=self._datasets[datasets_overview.selection[0]]).show()
        gui = pn.Column(datasets_overview, del_button, dataset_overview)
        
        def update_on_table_selection(selection):
            nonlocal dataset_overview
            old_active = dataset_overview.active
            dataset_overview = DatasetOverview(data=self._datasets[datasets_overview.selection[0]]).show()
            dataset_overview.active = old_active
            gui[-1] = dataset_overview
        datasets_overview.param.watch(update_on_table_selection, 'selection')
        
        def delete_entry(clicks):
            nonlocal datasets_overview
            nonlocal dataset_overview
            nonlocal gui
            self._datasets = [dataset for index, dataset in enumerate(self._datasets) if index not in datasets_overview.selection]
            datasets_overview = self.create_datasets_overview()
            datasets_overview.param.watch(update_on_table_selection, 'selection')
            gui[0] = datasets_overview
            
            if len(self._datasets) == 0:
                gui[-1] = None
            else:
                old_active = dataset_overview.active
                dataset_overview = DatasetOverview(data=self._datasets[datasets_overview.selection[0]]).show()
                dataset_overview.active = old_active
                gui[-1] = dataset_overview
        del_button.on_click(delete_entry)
        
        return gui

In [ ]:
test_dataset = aggregate_record_data(test_valid_records, test_class_map)
mdo = MultiDatasetOverview([test_dataset.iloc[:10], test_dataset.iloc[:50], test_dataset.iloc[:60], test_dataset])
mdo.show()

## Dataset filter

In [ ]:
#export
class DatasetFilter:
    """Creates a gallery with filter options to select subsets of the data. If the export_varialbe is set to a variable, that is a list a dict with the current selection (pd.Dataframe (key: data) and list of records (key: records)) will be append to the list."""
    def __init__(self, records, class_map=None, export_variable: Union[list, None] = None, height=500, width=500):
        self.records = records
        self.class_map = class_map
        self.export_variable = export_variable
        self.width = width
        self.height = height
    
        self.inner_width = int(0.97*width)
        self.data = aggregate_record_data(records, class_map)
            
        self.filters = []
    
    @staticmethod
    def _get_mask_from_range(values, selections):
        return (values >= selections[0]) & (values < selections[1])
    
    def show(self):
        """Creates a gallery with filter options to select subsets of the data. If the export_varialbe is set to a variable, that is a list a dict with the current selection (pd.Dataframe (key: data) and list of records (key: records)) will be append to the list."""
        # generate bbox filters
        options = self.data["label"].unique()
        options.sort()
        bbox_class_filter = pnw.MultiSelect(name="Class", options=options.tolist(), value=options.tolist())
        bbox_num_annotations_filter = generate_range_filter(self.data["num_annotations"], "Num. Annotations", steps=self.data["num_annotations"].max()+3)
        bbox_area_filter = generate_range_filter(self.data["area"], "Area", steps=10)
        bbox_width_filter = generate_range_filter(self.data["bbox_width"], "Bbox width", steps=10)
        bbox_height_filter = generate_range_filter(self.data["bbox_height"], "Bbbox height", steps=10)
        bbox_ratio_filter = generate_range_filter(self.data["bbox_ratio"], "Ratio", steps=10)
        annotation_filters = pn.Card(bbox_class_filter, bbox_num_annotations_filter, bbox_area_filter, bbox_width_filter, bbox_height_filter, bbox_ratio_filter, header="<h2>Bbox Filters</h2>", width=int(0.98*self.width), collapsed=True)

        # generate image filters
        file_creation_modification_time_filter = generate_creation_modification_time_filter(self.data, width=int(0.99*self.width))
        file_width_filter = generate_range_filter(self.data["width"], "Width", steps=10, width=int(0.99*self.width))
        file_height_filter = generate_range_filter(self.data["height"], "Height", steps=10, width=int(0.99*self.width))
        file_filters = pn.Card(file_creation_modification_time_filter, file_width_filter, file_height_filter, header="<h2>Image Filters</h2>", width=int(0.98*self.width), collapsed=True)

        filters = pn.Card(pn.Column(file_filters, annotation_filters), header="<h1>Filters</h1>", width=self.width, collapsed=True)

        def get_filtered_records_and_data():
            filtered_data = self.data[
                self.data["label"].isin(bbox_class_filter.value)
                & self._get_mask_from_range(self.data["num_annotations"], bbox_num_annotations_filter[0].value)
                & self._get_mask_from_range(self.data["area"], bbox_area_filter[0].value)
                & self._get_mask_from_range(self.data["bbox_width"], bbox_width_filter[0].value)
                & self._get_mask_from_range(self.data["bbox_height"], bbox_height_filter[0].value)
                & self._get_mask_from_range(self.data["bbox_ratio"], bbox_ratio_filter[0].value)
                & self._get_mask_from_range(self.data["creation_date"], file_creation_modification_time_filter[1].value)
                & self._get_mask_from_range(self.data["modification_date"], file_creation_modification_time_filter[2].value)
                & self._get_mask_from_range(self.data["width"], file_width_filter[0].value)
                & self._get_mask_from_range(self.data["height"], file_height_filter[0].value)
            ]
            filtered_records = [self.records[i] for i in filtered_data["record_index"].unique()]
            return filtered_records, filtered_data

        @pn.depends(
            bbox_class_filter.param.value, bbox_num_annotations_filter[0].param.value_throttled, bbox_area_filter[0].param.value_throttled, 
            bbox_width_filter[0].param.value_throttled, bbox_height_filter[0].param.value_throttled, bbox_ratio_filter[0].param.value_throttled,
            file_creation_modification_time_filter[1].param.value_throttled, file_creation_modification_time_filter[2].param.value_throttled, 
            file_width_filter[0].param.value_throttled, file_height_filter[0].param.value_throttled
        )
        def _gallery(
            classes_selection, num_annotation_selection, area_selection, width_selection, height_selection, ratio_selection, 
            image_creation_date_selection, image_modification_date_selection, image_with_selection, image_height_selection
        ):
            filtered_records, filtered_data = get_filtered_records_and_data()

            # update filter histograms
            bbox_num_annotations_filter[1] = histogram(self.data.groupby("id").count()["width"], bins=20, height=100, width=int(self.width*0.97), range=(self.data.groupby("id").count()["width"].min(), self.data.groupby("id").count()["width"].max()), remove_tools=True)
            bbox_area_filter[1] = histogram(filtered_data["area"], bins=20, height=100, width=int(self.width*0.97), range=(self.data["area"].min(), self.data["area"].max()), remove_tools=True)
            bbox_width_filter[1] = histogram(filtered_data["bbox_width"], bins=20, height=100, width=int(self.width*0.97), range=(self.data["bbox_width"].min(), self.data["bbox_width"].max()), remove_tools=True)
            bbox_height_filter[1] = histogram(filtered_data["bbox_height"], bins=20, height=100, width=int(self.width*0.97), range=(self.data["bbox_height"].min(), self.data["bbox_height"].max()), remove_tools=True)
            bbox_ratio_filter[1] = histogram(filtered_data["bbox_ratio"], bins=20, height=100, width=int(self.width*0.97), range=(self.data["bbox_ratio"].min(), self.data["bbox_ratio"].max()), remove_tools=True)

            if len(filtered_records) == 0:
                return None
            else:
                return gallery(filtered_records)

        if self.export_variable is not None:
            def export(event):
                filtered_records, filtered_data = get_filtered_records_and_data()
                self.export_variable.append({"data": filtered_data, "records": filtered_records})

            export_button = pnw.Button(name="Export")           
            export_button.on_click(export)

            return pn.Column(pn.Row(filters, align="center"), _gallery, export_button, align="center")
        else:
            return pn.Column(pn.Row(filters, align="center"), _gallery, align="center")

In [ ]:
dump_var = []
ds_filter = DatasetFilter(test_valid_records, test_class_map, export_variable=dump_var).show()
# trigger click event to test if record is added to dump_var
ds_filter[-1].clicks += 1
assert len(dump_var) == 1
assert len(dump_var[0]["records"]) == len(test_valid_records)
ds_filter

In [ ]:
dump_var[0]["data"]

## Dataset creator

In [ ]:
#export
class DatasetCreator:
    def __init__(self, records, class_map=None, height=500, width=500):
        self.height = height
        self.width = width
        self.records = records
        self.class_map = class_map
        self.data = aggregate_record_data(records, class_map)
        self.datasets = []
        self._filter = DatasetFilter(records, class_map, export_variable=self.datasets).show()
    
    def show(self):
        return pn.Column(self._filter)

In [ ]:
dsc = DatasetCreator(test_valid_records)

## Dahboard for comparing train and val dataset

In [ ]:
#export
class TrainValRecordsComparison:
    """
    Dashboard to compare the generated train and val records, to ensure the don't diverge to much from each other.
    """
    def __init__(self, train_records, val_records, class_map=None):
        self.train_records = train_records
        self.val_records = val_records
        self.class_map = class_map
        self.data = self._aggregate_data()
    
    def _aggregate_data(self):
        """Aggregates data from train and val records into a dataframe"""
        train_data = aggregate_record_data(self.train_records)
        val_data = aggregate_record_data(self.val_records)
        train_data["type"] = "train"
        val_data["type"] = "val"
        full_data = pd.concat([val_data, train_data])
        # convert the label column to the corrosponding names if a class_map is present and shift the previous labels to a label_num column
        # use the label column for here on generically
        if self.class_map is not None:
            full_data["label_num"] = full_data["label"]
            full_data["label"] = full_data["label"].apply(self.class_map.get_id)
        return full_data
    
    def _collect_stat(self):
        """Calculates stats on the aggregated data for the two datasets sets and returns them as a dataframe."""
        train_stats = {}
        val_stats = {}
        deviation_stats = {}
        # collect stats
        grouping = self.data.groupby("type")
        train_group = grouping.get_group("train")
        val_group = grouping.get_group("val")
        train_label_fractions = train_group.value_counts("label") / train_group.shape[0]
        val_label_fractions = val_group.value_counts("label") / val_group.shape[0]
        for key, value in val_label_fractions.items():
            val_stats["label_fraction_"+key] = value
        for key, value in train_label_fractions.items():
            train_stats["label_fraction_"+key] = value
        val_stats["area_min"] = val_group["area"].min()
        val_stats["area_max"] = val_group["area"].max()
        val_stats["area_avg"] = val_group["area"].mean()
        val_stats["area_std"] = val_group["area"].std()
        train_stats["area_min"] = train_group["area"].min()
        train_stats["area_max"] = train_group["area"].max()
        train_stats["area_avg"] = train_group["area"].mean()
        train_stats["area_std"] = train_group["area"].std()
        
        # calculated deviations dict
        for key, value in train_stats.items():
            deviation_stats[key] = 1-val_stats[key]/value
        # convert the dicts to dfs
        train_stats = pd.DataFrame(train_stats, index=["train"])
        val_stats = pd.DataFrame(val_stats, index=["val"])
        deviation_stats = pd.DataFrame(deviation_stats, index=["deviation (1-val/train)"])
        data = pd.concat([train_stats, val_stats, deviation_stats]).T
        data = data.round(2)
        data = data.reset_index()
        data["index"] = data["index"].apply(lambda x: x.replace("_", " ").title())
        return data
    
    def comparison_table(self, width=600, height=600):
        """Generates a table that compares stats between train and val datasets."""
        data = self._collect_stat()
        source = ColumnDataSource(data)
        
        columns = [TableColumn(field=filed, title=filed.replace("_", " ").title()) for filed in data.columns]
        data_table = DataTable(source=source, columns=columns, width=width, height=height)
        return data_table
    
    def area_histogram(self, width=500, height=500):
        """Generates area histogram for the train and val dataset."""
        return comparison_histogram_with_gui([group[1] for group in self.data.groupby("type")], hist_func=area_histogram, width=width, height=height)
    
    def show(self):
        table = self.comparison_table(width=600, height=250)
        area_histogram = self.area_histogram(width=600, height=500)
        
        return pn.Column(table, area_histogram)

In [ ]:
tvr_comp = TrainValRecordsComparison(test_train_records, test_valid_records, test_class_map)
tvr_comp.show()